# Lab03: Web Crawler (Continue) & Information Retrieval.

- MSSV: 1712429
- Họ và tên:Đỗ Văn Hiệp

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; từ `TODO` cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file notebook, file `t_data.txt` và file `raw_data` của các bạn (nếu file này kích thước lớn các bạn có thể chép link vào `link_data.txt`), nén thư mục `MSSV` này lại và nộp trên moodle.

**Nội dung bài tập**

Cài đặt một web crawler để thu thập dữ liệu từ: https://en.wikipedia.org/wiki/Web_mining.

## Nội dung bài tập

Cài đặt một Web crawler đơn giản bắt đầu từ URL: https://en.wikipedia.org/wiki/Web_mining, tìm liên kết và thu thập dữ liệu trong HTML tại URL này sau đó lặp lại với các URL vừa tìm được.


In [1]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string
import pickle

## 1. Thu thập đường dẫn

- Robot.txt:  https://en.wikipedia.org/robots.txt
- **Bước 1**: Thu thập đường dẫn từ https://en.wikipedia.org/wiki/Web_mining. Lưu trữ vào một danh sách `url_list`. 
- **Bước 2**: Lặp lại bước 1 cho các đường dẫn trong `url_list` (**lưu ý:** kiểm tra các đường dẫn vừa thu được đã nằm trong `url_list` hay không?). Dừng khi đã thu thập được 200 URLs.

In [2]:
def get_urls(url):
    r = requests.get(url)
    # TODO
    # Lấy các url nằm trong trang web của url này, lưu lại vào biến urls
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', r.text)
    urls_new=[]
    for i in urls:
        if (i not in urls_new):
            urls_new.append(i)
    return urls_new

def get_urls_recursive(start_url, limit):
    urls = [start_url]
    for url in urls:
        # TODO
        # Lấy các url nằm trong trang web của url này, lưu lại vào biến new_urls
        # Với mỗi url mới trong new_urls:
        #   Nếu nó chưa nằm trong urls thì thêm nó vô  
        # Nếu kích thước của urls vượt quá limit thì dừng và xóa phần dư thừa
        new=get_urls(url)# lây  tất cả các url của trang web lưu vào biến new
        for i in new: # kiểm tra các url trong new có nằm trong list urls hay ko
            if i not in urls: # neeuw khong co
                urls.append(i)
        if len(urls)>200:
            break
    return urls
url_list = get_urls_recursive('https://en.wikipedia.org/wiki/Web_mining', 200)
for i in url_list:
    print(i)

https://en.wikipedia.org/wiki/Web_mining
https://en.wikipedia.org/w/index.php?title=Web_mining&amp;action=edit
https://en.wikipedia.org/w/index.php?title=Web_mining&amp;action=edit&amp;section=
https://archive.org/details/webinformationsy00ngua
https://archive.org/details/webinformationsy00ngua/page/n33
https://archive.org/details/electroniccommer00bauk_698
https://archive.org/details/electroniccommer00bauk_698/page/n176
https://archive.org/details/webminingapplica0000scim/page/282
http://alexandria.tue.nl/repository/freearticles/612259.pdf
http://www.cis.unisa.edu.au/
https://docs.google.com/open?id=1nU1vrz-gBtSJk3bkb1ls_QuGX2nUPPemECPFlCx0C75MvmQdSqPci6LZWJYf
https://ui.adsabs.harvard.edu/abs/2000cs.......11033K
https://doi.org/10.1145%2F360402.360406
https://api.semanticscholar.org/CorpusID:60455
http://library.ifla.org/148/
https://doi.org/10.5195%2Fjmla.2019.650
http://soave.isti.cnr.it/%7Esilvestr/wp-content/uploads/2007/03/p63-baraglia.pdf
http://facweb.cs.depaul.edu/mobasher/cl

## 2. Thu thập dữ liệu
Thu thập dữ liệu từ `url_list`. Lưu trữ dữ liệu thu được vào dictionary data với keys là các từ, values gồm 2 phần tử: 
- `url_idx_list` với $idx \in \left[0,200\right) \cap \mathbb{N}$
- `frequency` 
    
Ví dụ: `data['at']=[url_idx_list,frequency]`:
- `url_idx_list`: danh sách các url mà trong dữ liệu của chúng (html document) chứa từ "at". 
- `frequency`: tần suất xuất hiện (số lần xuất hiện) của từ `at` trong dữ liệu của **tất cả đường dẫn thu được**.   

In [3]:
def text_filter(element):
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True
def wordList(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    text = soup.findAll(text=True)
    filtered_text = list(filter(text_filter, text)) # list của các chuỗi
    word_list = []
    
    for i in range(0,len(filtered_text)):
        if filtered_text[i] in string.punctuation:# kiểm tra xem có trong list kí tự hay không
            filtered_text[i]= filtered_text[i].replace(filtered_text[i]," ")#replace " " # thay thế bằng khoảng trắng
    for i in range(0,len(filtered_text)):
        list_w_split=[]
        list_w_split=filtered_text[i].split(" ")# tách các chuỗi thành các câu lưu vào word_list
        for j in range(0,len(list_w_split)):
            word_list.append(list_w_split[j])
    for i in word_list:
        if i in string.punctuation:# check word_list thêm một lần nữa, nếu có khoảng trắng thì remove
            word_list.remove(i)
    return word_list

def read_url(url, url_idx, data):
    word_list = wordList(url)
    
    # TODO
    # Với mỗi từ w trong word_list:
    #   Nếu w chưa có trong data thì khởi tạo data[w] = [[url_idx], 1]
    #   Ngược lại thì thêm url_idx vào data[w][0] (nếu chưa có) và tăng data[w][1] lên 1 đơn vị
    for i in range(0,len(word_list)) :
        if word_list[i] not in data.keys():
            data[word_list[i]]=[[url_idx],1]
            
            
        else:
            #check url
            if url_idx in data[word_list[i]][0]:
                data[word_list[i]][0].append(i)
                
            data[word_list[i]][1]=data[word_list[i]][1]+1

In [4]:
data = {}
#for url_index, url in enumerate(url_list, 1):
    #read_url(url, url_index, data)
for i in url_list:
    read_url(i,1,data)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


ConnectionError: HTTPConnectionPool(host='soave.isti.cnr.it', port=80): Max retries exceeded with url: /~silvestr/wp-content/uploads/2007/03/p63-baraglia.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000026B77A499E8>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

## 3. Tiền xử lý
Loại bỏ các item trong data mà key là các stopword.

**Ngữ liệu:** 

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
print(english_stopwords)

In [ ]:
# TODO
# Loại bỏ các key của biến data mà nằm trong danh sách english_stopwords
list_keys=data.keys()
for i in list(list_keys):
    if i in english_stopwords:# kiểm tra xem key có trong english top w ko
        data.pop(i)



## 4. Lưu trữ và biểu diễn dữ liệu
Sử dụng pickle lưu lại data với tên file raw_data.
### 4.1 Cơ sở dữ liệu giao tác:
Thông thường, các cơ sở dữ liệu giao tác được lưu trong flat files (các tập phẳng) thay vì trong một hệ cơ sở dữ liệu. Các item là các số nguyên không âm, mỗi giao tác tương ứng với một dòng các số nguyên phân tách nhau bằng khoảng trắng.
Ví dụ:

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 

32 69 

48 70 71 72 

39 73 74 75 76 77 78 79 

36 38 39 41 48 79 80 81 

82 83 84 

41 85 86 87 88 

39 48 89 90 91 92 93 94 95 96 97 98 99 100 101 

36 38 39 48 89 

39 41 102 103 104 105 106 107 108 

38 39 41 109 110 

39 111 112 113 114 115 116 117 118 

119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 

48 134 135 136 

39 48 137 138 139 140 141 142 143 144 145 146 147 148 149 

39 150 151 152 

38 39 56 153 154 155 

In [ ]:
with open('raw_data', 'wb') as f:
    pickle.dump(data,f)
    # TODO

### 4.2 Xuất dataset
Lưu một cơ sở dữ liệu giao tác (transactional database) vào file t_data.txt: 
- Các item tương ứng với url_idx
- Mỗi transaction tương ứng với một từ.

In [ ]:
with open('t_data.txt', 'w') as f:
    for word, (url_list, freq) in data.items():
        print(*url_list, file=f)

## 5. Truy vấn and, or, not
Ví dụ: 
- Truy vấn `and` câu `web mining`: trả về đường dẫn tới các trang web có cả 2 từ web và từ mining. 
- Truy vấn `or` câu `web mining`: trả về đường dẫn tới các trang web có từ web hoặc từ mining.
- Truy vấn `not` câu `web mining`: trả về đường dẫn tới các trang không có cả từ web và từ mining.

*GỢI Ý: TÁCH CÂU TRUY VẤN THÀNH CÁC TỪ TƯƠNG TỰ PHƯƠNG PHÁP LÀM Ở LAB02.*

In [5]:
ret=[]
def andRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have all terms: urls
    ### if len(ret)==0:  return urls
    ### else update ret with urls: intersection of ret and urls 
    terms=sentence.split(" ")# split query into terms
    for j in url_list: # duyệt qua tất cả các url
        flag=0
        for i in terms:# duyệt qua tất cả các terms trong sentence 
            check=j.find(i)# tìm kiếm xem trong url có terms không
            if(check==-1):# nếu không có 
                flag=0
                break# thoát khỏi vòng lặp
            else:
                flag=1#đổi biến flag
        if(flag==1):# nếu có tất cả các terms trong urls thì thêm url vào ret
            ret.append(j)# thêm url có tất cả các terms vào ret
        #nếu thoát khỏi vòng lặp kiểm tra len(ret=0)
    if len(ret)==0:
        return ret
    else:
        return ret
            
            
            
        


def orRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have all terms: urls
    ### find urls have at least 1 term: urls
    ### update ret with urls: extend ret with urls
    terms=sentence.split(" ")# split query into terms
    for i in url_list:
        for j in terms:
            check=i.find(j)# tìm trong urls xem có terms không
            if(check!=-1):#nếu có
                ret.append(i)# them urls vào res
                break #thoát
    return ret
            
            
    

def notRetrieval(ret, sentence):
    '''Parameters
    -----------------------
    ret: url_list
    sentence: query'''
    # TODO
    ### split sentence (separator ' ') into terms
    ### find urls have at least 1 term: urls
    ### update ret with urls: remove urls from ret 
    terms=sentence.split(" ")# split query into terms
    for j in url_list:# duyệt qua tất cả các url
        flag=0
        for i in terms:# duyệt qua tất cả các terms trong sentence 
            check=j.find(i)# tìm kiếm xem trong url có terms không
            if(check!=-1):# nếu  có các term
                flag=0
                break# thoát khỏi vòng lặp
            else:
                flag=1#đổi biến flag
        if(flag==1):# nếu  không có tất cả các terms trong urls thì thêm url vào ret
            ret.append(j)# thêm url có tất cả các terms vào ret
        
    
    return ret
    
        

In [ ]:

res=andRetrieval([], 'web mining')
print(" andretrieval")
for i in res:
    print(i)
orres=orRetrieval([], 'web mining')
print(" orretrieval")
for i in orres:
    print(i)
notres=notRetrieval(url_list, 'web mining')
print("notretrieval")
for i in notres:
    print(i)

 andretrieval
https://archive.org/details/webminingapplica0000scim/page/282
https://www.researchgate.net/profile/Magdalini_Eirinaki/publication/220169917_Web_mining_for_web_personalization/links/0912f51379b8daa82a000000.pdf
http://webmining.spd.louisville.edu/wp-content/uploads/2014/05/Nasraoui_WebKDD03_web_recomm.pdf
http://webmining.spd.louisville.edu/wp-content/uploads/2014/05/Nasraoui-IFSA-99-mining-web-access-logs.pdf
http://webmining.spd.louisville.edu/wp-content/uploads/2014/05/FINAL-Nasraoui-WWW-Personalization.pdf
https://archive.org/services/img/webminingapplica0000scim
https://archive.org/details/webminingapplica0000scim
https://archive.org/services/img/webminingapplica0000scim/full/pct:200/0/default.jpg
https://twitter.com/intent/tweet?url=https://archive.org/details/webminingapplica0000scim&amp;via=internetarchive&amp;text=Web+mining+%3A+applications+and+techniques+%3A+Scime%2C+Anthony%2C+1952-+%3A+Free+Download%2C+Borrow%2C+and+Streaming+%3A+Internet+Archive
https://www.f